## Monte Carlo Simulations

### A Small Example

Let's try to simulate

\begin{align*}
  \begin{aligned}
    x_{k+1} = \begin{bmatrix}
      0.5 & 0 \\
      0 & 1.5
    \end{bmatrix} x_{k} + \begin{bmatrix}
      1 \\ 1
    \end{bmatrix} u_{k} + n_{k} \,,
  \end{aligned}
\end{align*}

with the controller $u_{k} = \begin{bmatrix}0.16 & -1.96\end{bmatrix} x_{k}$ and standard, normal noise $n_{k}$.

#### Function Definition

Since Julia is JIT-compiled language, writing a function is almost always beneficial (for efficiency purposes).

#### References

Parallelization tips borrowed from Julia's [documentation](http://docs.julialang.org/en/stable/manual/parallel-computing/).

In [1]:
function simulate_serial!(x; x₀ = zeros(2))
    A = [0.5 0; 0 1.5]
    B = [1; 1]
    K = [0.16 -1.96]
    Ã = A + B*K
    for n = 1:size(x,3)
        x[:,1,n] = x₀
        for k = 2:size(x,2)
            x[:,k,n] = Ã*x[:,k-1,n] + randn(2)
        end
    end
    x
end

simulate_serial! (generic function with 1 method)

Run once to compile

In [2]:
x = zeros(2,100,1)
x₀ = [50; 100]
simulate_serial!(x, x₀ = x₀);

In [3]:
N = 100000 # Monte Carlo simulations
x = zeros(2,100,N)

@time simulate_serial!(x, x₀ = x₀);

  5.196376 seconds (99.10 M allocations: 4.576 GB, 6.83% gc time)


In [4]:
x[:,:,1]

2×100 Array{Float64,2}:
  50.0  -162.283   -29.1507   -5.96011  …  -1.5106    -2.20254   -0.9919  
 100.0   -39.5975   -7.11805  -1.15788      0.642435   0.133709  -0.656561

In [5]:
x[:,:,2]

2×100 Array{Float64,2}:
  50.0  -163.12    -34.5176  -4.95984  …  1.80269  -6.10451  -0.186694
 100.0   -37.9803   -8.6803  -2.66838     4.71145  -2.05684   0.815923

Now, let's try to parallelize it among 4 workers.

In [6]:
workspace()
addprocs(4) # Add 4 workers

4-element Array{Int64,1}:
 2
 3
 4
 5

In [7]:
function simulate_parallel!(x; x₀ = zeros(2))
    A = [0.5 0; 0 1.5]
    B = [1; 1]
    K = [0.16 -1.96]
    Ã = A + B*K
    @sync @parallel for n = 1:size(x,3)
        x[:,1,n] = x₀
        for k = 2:size(x,2)
            x[:,k,n] = Ã*x[:,k-1,n] + randn(2)
        end
    end
    x
end

simulate_parallel! (generic function with 1 method)

In [8]:
N = 100000 # Monte Carlo simulations
x = zeros(2,100,N)
x₀ = [50; 100]

@time simulate_parallel!(x, x₀ = x₀);

  8.261979 seconds (2.37 M allocations: 102.886 MB, 1.34% gc time)


In [9]:
x[:,:,1]

2×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [10]:
x[:,:,2]

2×100 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [12]:
x = SharedArray(Float64, (2,100,N))

@time simulate_parallel!(x, x₀ = x₀);

  2.728109 seconds (1.50 k allocations: 49.063 KB)


In [13]:
x[:,:,1]

2×100 Array{Float64,2}:
  50.0  -164.215   -34.0463   -4.14364   …  2.59845  -5.24263  -0.723721
 100.0   -38.6228   -9.28956  -0.998996     2.69325  -1.161    -0.186591

In [14]:
x[:,:,2]

2×100 Array{Float64,2}:
  50.0  -161.633   -31.5969   -1.47357   …  5.32796   2.45212   0.950797
 100.0   -38.1052   -9.89967   0.232877     0.292643  0.755201  1.24397 

In [15]:
@everywhere function mychunk(q::SharedArray)
    idx = indexpids(q)
    idx == 0 && return 1:0 # master does not have any work
    nchunks = length(procs(q))
    splits = [round(Int,s) for s in linspace(0, size(q,3), nchunks + 1)]
    return splits[idx]+1:splits[idx+1]
end

In [16]:
@everywhere function simulate_chunk!(x; x₀ = zeros(2))
    A = [0.5 0; 0 1.5]
    B = [1; 1]
    K = [0.16 -1.96]
    Ã = A + B*K
    for n in mychunk(x)
        x[:,1,n] = x₀
        for k = 2:size(x,2)
            x[:,k,n] = Ã*x[:,k-1,n] + randn(2)
        end
    end
    x
end

In [17]:
function simulate_shared!(x; x₀ = zeros(2))
    @sync for p in procs(x)
        @async remotecall_wait(simulate_chunk!, p, x; x₀ = x₀)
    end
end

simulate_shared! (generic function with 1 method)

In [19]:
x = SharedArray(Float64, (2,100,N))

@time simulate_shared!(x, x₀ = x₀)

  2.734287 seconds (759 allocations: 58.094 KB)


In [20]:
x[:,:,1]

2×100 Array{Float64,2}:
  50.0  -163.797   -33.2074  -7.40186   …  0.995661  -0.318155  -3.25702
 100.0   -38.2058   -8.221   -0.976372     0.812627   1.07233    1.51922

In [21]:
x[:,:,2]

2×100 Array{Float64,2}:
  50.0  -162.108  -34.6556   -4.18028  …  3.55388  -0.50931    0.137739
 100.0   -37.14    -9.81649  -1.48247     1.94443  -0.369575  -2.38181 